In [31]:
import pandas as pd
import random
import numpy as np
import openai
import csv
import json

In [ ]:
file_path = r"C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\chat_messages_202409051704.json"

with open(file_path, "rb") as f:
    json_file = json.load(f)

df = pd.DataFrame(list(json_file.values())[0])

In [222]:
df['timestamp'] = pd.to_datetime(df['created_at'], format='%Y-%m-%dT%H:%M:%S.%fZ', utc=True)
df['month'] = df['timestamp'].dt.month

In [224]:
df.head()

,message_id,user_query,created_at,timestamp,month
0,15fffc21-5625-4a68-a3a6-4652808e18b0,Give me a trade to make For the last hour of t...,2024-06-14T06:39:36.157Z,2024-06-14 06:39:36.157000+00:00,6
1,4efcd440-86c2-44ac-8d6f-3e25f23c29e6,Give me 3 good forex trade setups from out of ...,2024-06-14T06:39:45.901Z,2024-06-14 06:39:45.901000+00:00,6
2,77681cc3-7f84-40a4-87e7-0bd5735e2155,give me an in the money options trade with a s...,2024-06-14T06:39:59.462Z,2024-06-14 06:39:59.462000+00:00,6
3,7f450bb0-9fca-43dc-bf98-69951a211830,what is the top 3 best call options to buy wit...,2024-06-14T06:40:18.882Z,2024-06-14 06:40:18.882000+00:00,6
4,81879c99-afec-49ce-9e25-c2ebd8f7edcb,Given the current market conditions... what is...,2024-06-14T06:40:32.365Z,2024-06-14 06:40:32.365000+00:00,6


In [299]:
docs_jun_jul = df[(df['month'] == 5) | (df['month'] == 6)]['user_query'].tolist()

In [301]:
len(docs_jun_jul)

115540

In [303]:
new_docs = docs_jun_jul[:300000]
len(new_docs)

115540

# New messages classification - prompt engineering

#### --async processing

In [269]:
from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
MAX_CONCURRENCY = 32

In [271]:
semaphore = asyncio.Semaphore(MAX_CONCURRENCY)
model = "gpt-4o-mini" 
api_key="sk-proj-...z"
async_client = AsyncOpenAI(
    api_key=api_key,
)


In [273]:
topics = [
    "Options Trading Strategies",
    "Stock Market Analysis",
    "Industry & Sector Trend",
    "Trading Terminology & Tools",
    "Technical Indicators & Analysis",
    "Non-Financial/Unrelated Topics",
    "Market Sentiment & Trends",
    "Earnings & Financial Reports",
    "Cryptocurrency",
    "Forex"
]

topics_guidelines = {
    "Options Trading Strategies": "Messages about strategies, tips, or questions related to trading options.",
    "Stock Market Analysis": "Messages that inquire about stock price, discuss stock market, stock price movements, market predictions, or stock performance.",
    "Industry & Sector Trend": "Messages discussing trends or changes within specific industries or market sectors.",
    "Trading Terminology & Tools": "Messages asking about or discussing trading terminology, algo trading, codes, tools, or platforms.",
    "Technical Indicators & Analysis": "Messages focused on technical analysis, including indicators, charts, and support/resistance levels etc.",
    "Non-Financial/Unrelated Topics": "Messages that do not relate to financial topics at all.",
    "Market Sentiment & Trends": "Messages discussing general market sentiment, investor moods, or broad market trends.",
    "Earnings & Financial Reports": "Messages related to earnings reports, financial statements, or company financial data.",
    "Cryptocurrency": "Messages about cryptocurrency, including Bitcoin, Ethereum, and other digital currencies.",
    "Forex": "Messages about foreign exchange markets, currency pairs, or forex trading."
}

### define the prompt
prompt = f"""\
You are an expert in financial and trading topics. You will be given a list of 10 predefined topics:
Here are the topic list:
{','.join(topics)}

Your task is to assign the message to the most representative topic from the topic list given above. 

**Instructions:**
- Your response **must be exactly one of the predefined topics** without any additional text, punctuation, or special characters;
- If a message is ambiguous or lacks detail, infer the most likely topic based on common financial and trading contexts.
- **Only return "Non-Financial/Unrelated Topics" if the message is clearly irrelevant or does not pertain to financial, trading, or market-related subjects at all**.
- do not provide any other content, explanations, or answers to message, nor to repeat the message;
- **Strictly limit your response to the topic name only**, ensuring it is no longer than 10 words;


Here are some examples to guide you:

1. **Message**: New tools for automated trading and their advantages
   **Response**: Trading Terminology & Tools

2. **Message**: The latest financial report from a major company.
   **Response**: Earnings & Financial Reports

3. **Message**: What stock should I buy tomorrow under $5
   **Response**: Stock Market Analysis
   
4. **Message**: ?
   **Response**: Non-Financial/Unrelated Topics
   
5. **Message**: spy
   **Response**: Stock Market Analysis

5. **Message**: when is the cpi data
   **Response**: Market Sentiment & Trends
   
"""
prompt +=  '**Messages**: "{message}" Response:'

#- consider the **dominant topic** of the message, even if multiple topics seem relevant, choose the most specific and direct match;


# For each message, consider the following guidelines:
# {'\n'.join([f"{topic}: {guideline}" for topic, guideline in topics_guidelines.items()])}

In [275]:
async def process_review(text):
    async with semaphore:
        try:
            input = [{"role": "user", "content": prompt.format(message=text)}]
            
            res = await async_client.chat.completions.create(
                model=model,
                messages=input,
            )
            result = res.choices[0].message.content

            if result not in set(topics):
                result = "Non-Financial/Unrelated Topics"
            return result
        
        except Exception as e:
            print(f"Error: {e}. Failed {text}")
            return ""

In [277]:
async def main(docs):
    tasks = []
    for i, text in enumerate(docs):  # Use enumerate to get both index and text
        tasks.append((i, process_review(text)))  # Store index and task together

    # Use list comprehension to gather results and maintain index association
    res = await tqdm.gather(*(task[1] for task in tasks))

    final_output = [res[i] for i in range(len(res))]
    return final_output
# asyncio.run(main())

In [279]:
sample_messages = [
    'jblu trade',
    'sbux',
    'What is the CGAR for Cadence for the last year?',
    'What is the highest yielding index fund ',
    'what is the lowest price of jakk in the past year',
    'Can you make it 10 stocks',
    'can you find me a different stock',
    'analyse sndx',
    'Is ctxr still trading? Did tenk merge with them? Is ctor trading?'
    'mu short term target price',
    'Assuming 8% annual growth and an investment of $100 per month, how long would it take to reach one million dollars?',
    'What if the growth rate was 12%',
    '30 day dividen payout ',
    'Give me stocks which will ultimately make me 1k plus per day in total ',
    'Show a chart of Lockheed',
    'Analyze DHT',
    'when is the cpi data',
    'What stock should I buy tomorrow under $5',
    'Can you help me find stock for day trading',
    'How long should I hold adma',
    'What are the rules every trader should follow to become profitable ',
    'I need a potiential 500% return today ',
    'if i pull out of SOL at 145 as suggested what should be my next 30 minute trade?'
]

In [281]:
await main(sample_messages)

100%|██████████| 22/22 [00:00<00:00, 25.31it/s]


['Stock Market Analysis',
 'Stock Market Analysis',
 'Earnings & Financial Reports',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Non-Financial/Unrelated Topics',
 'Market Sentiment & Trends',
 'Earnings & Financial Reports',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Market Sentiment & Trends',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Stock Market Analysis',
 'Trading Terminology & Tools',
 'Stock Market Analysis',
 'Cryptocurrency']

In [305]:
mini_4o_output = await main(new_docs)
final_output_array = np.array(mini_4o_output)
#np.save(f'C:\\Users\\Liu Shi Peng\\Documents\\feb_jul_topic_classification\\batch_august.npy', final_output_array)

 89%|████████▉ | 103369/115540 [34:49<1:52:45,  1.80it/s]

Error: Request timed out.. Failed With the stop loss order I’ll set at 3.90


100%|██████████| 115540/115540 [45:15<00:00, 42.55it/s]  


In [287]:
final_output_array.shape

(89326,)

In [289]:
np.unique(final_output_array)

array(['Cryptocurrency', 'Earnings & Financial Reports', 'Forex',
       'Industry & Sector Trend', 'Market Sentiment & Trends',
       'Non-Financial/Unrelated Topics', 'Options Trading Strategies',
       'Stock Market Analysis', 'Technical Indicators & Analysis',
       'Trading Terminology & Tools'], dtype='<U31')

In [291]:
mini_4o_df = pd.DataFrame(final_output_array, columns = ['topic'])
mini_4o_df['message_id'] =df['message_id']
mini_4o_df['user_query'] =df['user_query']
mini_4o_df['date'] =df['created_at']
mini_4o_df.head()
mini_4o_df.to_csv('C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\topic_4o_mini_jul.csv')

#### Topic Analysis from Feb - Aug

In [120]:
df1 = pd.read_csv('C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\topic_4o_mini_aug.csv')
df2 = pd.read_csv('C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\db_chat_messages_with_topic_feb_jul.csv')

C:\Users\Liu Shi Peng\AppData\Local\Temp\ipykernel_11228\2241853323.py:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\db_chat_messages_with_topic_feb_jul.csv')


In [ ]:
print('df1_shape:', df1.shape)
print('df12_shape:', df2.shape)

In [ ]:
df2_limited = df2[['message_id', 'user_query_x','topic_y','created_at']].rename(columns =  {'user_query_x': 'query', 'topic_y':'topic'})

df1_limited = df1[['message_id', 'user_query','topic','date']].rename(columns =  {'user_query': 'query', 'date':'created_at'})

In [178]:
concat_df = pd.concat([df1_limited, df2_limited], axis = 0)
concat_df.shape

(430564, 4)

In [182]:
concat_df.head()

,message_id,query,topic,created_at
0,1714edcc-4b55-4c55-8fb2-23412712218a,Give me a buying entry level for SPY based on ...,Technical Indicators & Analysis,2024-09-03T12:00:11.310Z
1,c718635a-a6d3-40d0-82e2-30051baf88a5,Then what's after that,Non-Financial/Unrelated Topics,2024-09-03T12:00:15.195Z
2,08b4403b-8d52-413f-9480-a66d84eb2795,It's asking for my strategy with the vaxcyte,Options Trading Strategies,2024-09-03T12:00:29.049Z
3,78f96b4c-22c1-4ece-b1a5-b5c98bfd8003,Can you tell me a joke that involves a jew,Non-Financial/Unrelated Topics,2024-09-03T12:00:42.667Z
4,35157a27-d5d5-4510-a7a0-a58f358203bd,Dammi un segnale con esecuzione di mercato ist...,Forex,2024-09-03T12:01:10.678Z


In [188]:
concat_df['create_date'] = concat_df['created_at'].dt.month

AttributeError: Can only use .dt accessor with datetimelike values

#### rerun, data patch for failed LLM topic generation

In [463]:
aug_array = final_output_array
topic_df = pd.DataFrame(aug_array, columns = ['id','topic_name'])

In [469]:
conn = sqlite3.connect('jupyter_sql_tutorial.db')
topic_df.to_sql('topic_aug', conn, if_exists='replace')

37859

In [479]:
pd.read_sql("""select topic_name, count(*) from topic_aug group by 1""", con = conn)

,topic_name,count(*)
0,,153
1,"""Cryptocurrency""",1
2,"""I'm looking for a trading strategy that uses ...",1
3,"""Options Trading Strategies""",1
4,"""Stock Market Analysis""",1
5,"""Technical Indicators & Analysis""",1
6,"""Trading Terminology & Tools""",3
7,1. Forex \n2. Stock Market Analysis,1
8,1. Stock Market Analysis\n2. Stock Market Anal...,1
9,"Commodity Market Analysis is not in the list, ...",1


In [489]:
sql = pd.read_sql(
    """
    select id
    from (
        select id, topic_name, count(id) over (partition by topic_name) as cnt 
        from topic_aug ) t
    where t.cnt < 200
    """, con = conn)
list_id_to_rerun = sql['id'].tolist()
len(list_id_to_rerun)

180

In [491]:
list_to_rerun = []
for i in list_id_to_rerun:
    list_to_rerun.append((int(i), new_docs[int(i)]))
len(list_to_rerun)

180

In [493]:
async def main_with_input(list_to_rerun):
    tasks = []
    for i, text in list_to_rerun:  
        tasks.append((i, process_review(text)))  # Store index and task together

    # Use list comprehension to gather results and maintain index association
    res = await tqdm.gather(*(task[1] for task in tasks))

    final_output = [(tasks[i][0], res[i]) for i in range(len(res))]
    return final_output

In [495]:
list_to_rerun_topic = await main_with_input(list_to_rerun)

100%|██████████| 180/180 [01:47<00:00,  1.67it/s]


In [509]:
len(list_to_rerun_topic)

180

In [501]:
np.save(f'C:\\Users\\Liu Shi Peng\\Documents\\feb_jul_topic_classification\\batch_list2_rerun.npy', list_to_rerun_topic)

In [511]:
final_batch_with_patch = aug_array

for i, text in list_to_rerun_topic:
    if text not in topics: # to correct those not assigned proper topic by LLM to 
        final_batch_with_patch[int(i)][1] = 'Non-Financial/Unrelated Topics'
    else: 
        final_batch_with_patch[int(i)][1] = text

In [513]:
topic_df_with_patch = pd.DataFrame(final_batch_with_patch, columns = ['id', 'topic'])
topic_df_with_patch.shape

(37859, 2)

In [515]:
df.shape

(37859, 2)

In [519]:
final_df_with_topic = pd.concat([df, topic_df_with_patch['topic'] ], axis = 1 )
final_df_with_topic.shape

(37859, 3)

In [521]:
final_df_with_topic['topic'].value_counts()

topic
Options Trading Strategies         13142
Stock Market Analysis               9972
Technical Indicators & Analysis     4516
Non-Financial/Unrelated Topics      4240
Cryptocurrency                      1600
Trading Terminology & Tools         1535
Market Sentiment & Trends           1008
Forex                                800
Earnings & Financial Reports         663
Industry & Sector Trend              383
Name: count, dtype: int64

In [523]:
final_df_with_topic.to_csv("C:\\Users\\Liu Shi Peng\\Documents\\feb_jul_topic_classification\\tgpt_by_aug_13_topic_mapping_by_llama.csv")

#### 4omini vs llama result 

In [738]:
llama_df = pd.read_csv('C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\tgpt_by_aug_13_topic_mapping_by_llama.csv')
llama_df.shape

(37859, 4)

In [740]:
llama_df.columns

Index(['Unnamed: 0', 'message_id', 'user_query', 'topic'], dtype='object')

In [742]:
llama_df = llama_df.rename(columns = {'topic': 'topic_llama'})

In [744]:
llama_df.columns

Index(['Unnamed: 0', 'message_id', 'user_query', 'topic_llama'], dtype='object')

In [746]:
# llama_df['message_id'] = llama_df['message_id'].astype(str)
# mini_4o_df['message_id'] = mini_4o_df['message_id'].astype(str)

In [748]:
compare_df = llama_df.merge(mini_4o_df, how = 'inner', left_on = 'message_id', right_on = 'message_id')
compare_df.head()

,Unnamed: 0,message_id,user_query_x,topic_llama,id,topic,user_query_y
0,0,56284ad7-adfc-4ba9-b30f-16b51f3a8fac,Please find me 12 stocks with these filters: a...,Stock Market Analysis,0,Options Trading Strategies,Please find me 12 stocks with these filters: a...
1,1,458f9dcd-0b8e-436a-b842-8a9239c34480,Can you give me an options trade that can go 1...,Options Trading Strategies,1,Options Trading Strategies,Can you give me an options trade that can go 1...
2,2,7ffecec9-9832-46ae-81c0-48e5040ca3ee,Spy options,Options Trading Strategies,2,Options Trading Strategies,Spy options
3,3,318651fc-4729-40b0-a6b9-ab4c8bd070b5,Please continue. If Bitcoin is up what pairs a...,Cryptocurrency,3,Cryptocurrency,Please continue. If Bitcoin is up what pairs a...
4,4,994726e7-8b9e-43d1-ba5f-6d18249f068c,sbux,Stock Market Analysis,4,Stock Market Analysis,sbux


In [750]:
delta = compare_df[compare_df['topic_llama'] != compare_df['topic']]
delta

,Unnamed: 0,message_id,user_query_x,topic_llama,id,topic,user_query_y
0,0,56284ad7-adfc-4ba9-b30f-16b51f3a8fac,Please find me 12 stocks with these filters: a...,Stock Market Analysis,0,Options Trading Strategies,Please find me 12 stocks with these filters: a...
5,5,a8058940-85d2-4610-a4a0-b86afc86250e,What is the percentage on the QQQ that the hig...,Options Trading Strategies,5,Technical Indicators & Analysis,What is the percentage on the QQQ that the hig...
6,6,4c0a854a-1b45-4846-b6e2-702d362a3a4a,"Best placement for a buy limit, stop loss and ...",Technical Indicators & Analysis,6,Options Trading Strategies,"Best placement for a buy limit, stop loss and ..."
10,10,46fc052e-f215-45ac-a1a0-556a81616f6a,Give me a trade for SBUX or CMG to take for th...,Options Trading Strategies,10,Stock Market Analysis,Give me a trade for SBUX or CMG to take for th...
11,11,0be2699c-1db2-490d-9aac-695013804516,Expires Friday,Options Trading Strategies,11,Non-Financial/Unrelated Topics,Expires Friday
...,...,...,...,...,...,...,...
4976,4976,539e5b29-6741-48e3-8634-85dccf160a15,What do you see in video closing at,Stock Market Analysis,4976,Non-Financial/Unrelated Topics,What do you see in video closing at
4980,4980,55a3a845-043c-4c08-a233-3498e67050db,Based on technical analysis which stock is lik...,Technical Indicators & Analysis,4980,Stock Market Analysis,Based on technical analysis which stock is lik...
4990,4990,7dc30513-680b-4a18-821b-df78f7918777,Is amd bullish,Stock Market Analysis,4990,Market Sentiment & Trends,Is amd bullish
4992,4992,df4cb716-75a0-44fa-828d-47e480024650,"How to inverts $10,000 to gain monthly passive...",Stock Market Analysis,4992,Non-Financial/Unrelated Topics,"How to inverts $10,000 to gain monthly passive..."


In [752]:
delta.to_csv('C:\\Users\\Liu Shi Peng\\Documents\\feb_2_aug_topic_classification\\delta_40_mini.csv')